# Машинное обучение, РЭШ

## [Практическое задание 3. Градиентный спуск своими руками](https://www.youtube.com/watch?v=dQw4w9WgXcQ)

### Общая информация
Дата выдачи: 17.11.2022

Дедлайн: 23:59MSK 24.11.2022

### О задании

В данном задании необходимо реализовать обучение линейной регрессии с помощью различных вариантов градиентного спуска.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.


### Формат сдачи
Задания загружаются на my.nes. Присылать необходимо ноутбук с выполненным заданием. 

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

**Оценка**: ...

## Реализация градиентного спуска

Реализуйте линейную регрессию с функцией потерь MSE, обучаемую с помощью:

**Задание 1 (1 балл)** Градиентного спуска;

**Задание 2.1 (2 балла)** Стохастического градиентного спуска + Batch SGD;

**Задание 2.2 (2 балла)** SGD Momentum;

**Бонусное задание (2 балл)** Adagrad, RMSProp, Adam;

Во всех пунктах необходимо соблюдать следующие условия:

* Все вычисления должны быть векторизованы;
* Циклы средствами python допускается использовать только для итераций градиентного спуска;
* В качестве критерия останова необходимо использовать (одновременно):

    * проверку на евклидовую норму разности весов на двух соседних итерациях (например, меньше некоторого малого числа порядка $10^{-6}$, задаваемого параметром `tolerance`);
    * достижение максимального числа итераций (например, 10000, задаваемого параметром `max_iter`).
* Чтобы проследить, что оптимизационный процесс действительно сходится, будем использовать атрибут класса `loss_history` — в нём после вызова метода `fit` должны содержаться значения функции потерь для всех итераций, начиная с первой (до совершения первого шага по антиградиенту);
* Инициализировать веса можно случайным образом или нулевым вектором. 


Ниже приведён шаблон класса, который должен содержать код реализации каждого из методов.

In [122]:
def pprint(*args, **kwargs):
    print(">>", *args, **kwargs)

In [134]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error


class LinearReg(BaseEstimator):
    def __init__(self, gd_type='stochastic', 
                 tolerance=1e-4, max_iter=1000, w0=None, eta=1e-2):
        """
        gd_type: 'full' or 'stochastic'
        tolerance: for stopping gradient descent
        max_iter: maximum number of steps in gradient descent
        w0: np.array of shape (d) - init weights
        eta: learning rate
        alpha: momentum coefficient
        """
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.w = None
        self.eta = eta
        self.loss_history = None # list of loss function values at each training iteration
        self.w_history = None # list of weights at each training iteration
    
    def fit(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: self
        """
        self.loss_history = []
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

        # initialize vector of weights
        self.w = self.w0.copy()
        self.w_history = [self.w.copy()]
        self.loss_history = [root_mean_squared_error(y, X @ self.w_history[-1])]

        for i in tqdm(range(self.max_iter)):
            self.w -= self.eta * self.calc_gradient(X, y)
            self.w_history.append(self.w.copy())
            self.loss_history.append(root_mean_squared_error(y, X @ self.w_history[-1]))

            pprint(f"step is {i} and loss is {self.loss_history[-1]}")

            if (np.linalg.norm(self.w_history[-1] - self.w_history[-2]) <= self.tolerance) or (i >= self.max_iter):
                break

        # transform list to np.array
        self.w_history = np.array(self.w_history)

        return self
    
    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass
    
    def calc_gradient(self, X, y):
        """
        X: np.array of shape (ell, d) (ell can be equal to 1 if stochastic)
        y: np.array of shape (ell)
        ---
        output: np.array of shape (d)
        """
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

        if self.gd_type == 'full':
            return 2 * X.T @ (X @ self.w - y) / y.shape[0]
        else:
            ...

    def calc_loss(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: float 
        """ 
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass

In [128]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
# test on example from seminar
n_features = 2
n_objects = 300

w_true = np.random.normal(size=(n_features, ))
X = np.random.uniform(-5, 5, (n_objects, n_features))
X *= (np.arange(n_features) * 2 + 1)[np.newaxis, :]  # for different scales
Y = X.dot(w_true) + np.random.normal(0, 1, (n_objects))
w_0 = np.random.uniform(-2, 2, (n_features))

In [135]:
temp_class2 = LinearReg(w0=w_0, gd_type='full', eta=1e-2, max_iter=100, tolerance=1e-6)
temp_class2.fit(X=X, y=Y)

 76%|███████▌  | 76/100 [00:00<00:00, 4262.73it/s]

>> step is 0 and loss is 7.695910567156675
>> step is 1 and loss is 6.344803110649455
>> step is 2 and loss is 5.311175679943845
>> step is 3 and loss is 4.479245888347971
>> step is 4 and loss is 3.7959667047001866
>> step is 5 and loss is 3.2314787284015383
>> step is 6 and loss is 2.765490719620739
>> step is 7 and loss is 2.3824822728780912
>> step is 8 and loss is 2.0698322972833485
>> step is 9 and loss is 1.8169280927625029
>> step is 10 and loss is 1.6146307831717996
>> step is 11 and loss is 1.4549103734937778
>> step is 12 and loss is 1.3306061112482896
>> step is 13 and loss is 1.235302223748868
>> step is 14 and loss is 1.1632985603309345
>> step is 15 and loss is 1.1096340558099378
>> step is 16 and loss is 1.0701129735932913
>> step is 17 and loss is 1.041297649802513
>> step is 18 and loss is 1.0204565506915575
>> step is 19 and loss is 1.0054771796334983
>> step is 20 and loss is 0.9947620313064355
>> step is 21 and loss is 0.9871243297288548
>> step is 22 and loss is 0

,gd_type,'full'
,tolerance,1e-06
,max_iter,100
,w0,array([-1.978... 1.69112072])
,eta,0.01


**Задание 3 (0 баллов)**
* Загрузите данные из домашнего задания 2 ([train.csv](https://www.kaggle.com/c/nyc-taxi-trip-duration/data));
* Разбейте выборку на обучающую и тестовую в отношении 7:3 с random_seed=0;
* Преобразуйте целевую переменную `trip_duration` как $\hat{y} = \log{(y + 1)}$.

In [11]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
# I decided to use already processed data
df = pd.read_csv("data/final_df_2nd_homework.csv")
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 26 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  str    
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  str    
 3   passenger_count     1458644 non-null  int64  
 4   pickup_longitude    1458644 non-null  float64
 5   pickup_latitude     1458644 non-null  float64
 6   dropoff_longitude   1458644 non-null  float64
 7   dropoff_latitude    1458644 non-null  float64
 8   store_and_fwd_flag  1458644 non-null  int64  
 9   log_trip_duration   1458644 non-null  float64
 10  weekday             1458644 non-null  str    
 11  month               1458644 non-null  int64  
 12  hour                1458644 non-null  int64  
 13  day_year_number     1458644 non-null  int64  
 14  if_anomaly1         1458644 non-null  bool   
 15  if_anomaly2         145864

In [12]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=0)

**Задание 4 (3 балла)**. Обучите и провалидируйте модели на данных из предыдущего пункта, сравните качество между методами по метрикам MSE и $R^2$. Исследуйте влияние параметров `max_iter` и `eta` на процесс оптимизации. Согласуется ли оно с вашими ожиданиями?

In [21]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
lin_ref = LinearReg(max_iter=10, gd_type='full')

**Задание 5 (6 балла)**. Постройте графики (на одной и той же картинке) зависимости величины функции потерь от номера итерации для всех реализованных видов стохастического градиентного спусков. Сделайте выводы о скорости сходимости различных модификаций градиентного спуска.

Не забывайте о том, что должны получиться *красивые* графики!

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

**Задание 6 (бонус) (0.01 балла)**.  Вставьте картинку с вашим любимым мемом в этот Jupyter Notebook